In [ ]:
import ollama

def generate_kpi_mistral(kpi: dict, model='mistral'):
    
    changes = []
    for metric, info in kpi.get("metrics", {}).items():
        pct = info.get("change", 0) * 100
        sign = "+" if pct >= 0 else ""
        changes.append(f"{metric}: {sign}{pct:.1f}%")
    metrics_str = "; ".join(changes)

    prompt = (
        f"Ты бизнес-аналитик, используешь деловой тон. Составь краткий отчёт (2–4 предложения) на русском языке для менеджера по KPI, "
        f"упоминая ключевую метрику (например, GMV или количество заказов) и её динамику.\n"
        f"В начале выводи краткое описание (например, Сильный рост по всем фронтам; " 
        f"смешанная динамика для неоднозначных изменений и т.д.): GMV растет, Заказы падают). "
        f"Если другие метрики сильно изменились, упомяни об этом. "
        f"Используй только исключительно предоставленные данные: {metrics_str}. "
        f"Сравниваем период {kpi['period']} и {kpi['comparison_period']}. "
    )
    
    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response['message']['content'].strip()

### Модель выбиралась исходя из требований по памяти в условиях ограниченных ресурсов, её корректной работы с русским языком и достаточной сложностью (Gemma:2b выдавала не очень хорошие результаты, а IlyaGusev/rut5_base_sum_gazeta выдавала отвратительные результаты)

### Промпт составлялся секционно по указаниям в ТЗ. Насколько мне известно, перенос строки помогает модели лучше разделять смысловые части, а указание делового тона перенесено вначало по интуитивным соображениям по аналогии с человеческим мышлением

In [2]:
# 1. Сильный рост по всем фронтам (День)
daily_strong_growth = {
  "period": "2023-11-15",
  "comparison_period": "2023-11-14",
  "metrics": {
    "GMV": {"value": 1625000, "change": 0.25}, # +25%
    "Заказы": {"value": 2400, "change": 0.20},  # +20%
    "Средний чек (AOV)": {"value": 677.08, "change": 0.042}, # +4.2%
    "Сессии": {"value": 60000, "change": 0.18},  # +18%
    "Конверсия (CR)": {"value": 0.04, "change": 0.017} # +1.7%
  }
}

# 2. Сильное падение по всем фронтам (День)
daily_strong_decline = {
  "period": "2023-11-16",
  "comparison_period": "2023-11-15",
  "metrics": {
    "GMV": {"value": 980000, "change": -0.22},  # -22%
    "Заказы": {"value": 1800, "change": -0.18},  # -18%
    "Средний чек (AOV)": {"value": 544.44, "change": -0.049},# -4.9%
    "Сессии": {"value": 48000, "change": -0.15},  # -15%
    "Конверсия (CR)": {"value": 0.0375, "change": -0.035}# -3.5%
  }
}

# 3. Смешанная динамика: GMV растет, Заказы падают (День) -> Рост за счет AOV
daily_mixed_gmv_up_orders_down = {
  "period": "2023-11-17",
  "comparison_period": "2023-11-16",
  "metrics": {
    "GMV": {"value": 1060000, "change": 0.082}, # +8.2%
    "Заказы": {"value": 1710, "change": -0.05},  # -5%
    "Средний чек (AOV)": {"value": 619.88, "change": 0.139}, # +13.9%
    "Сессии": {"value": 49000, "change": 0.021},  # +2.1%
    "Конверсия (CR)": {"value": 0.0349, "change": -0.069} # -6.9%
  }
}

# 4. Смешанная динамика: GMV падает, Заказы растут (День) -> Падение за счет AOV
daily_mixed_gmv_down_orders_up = {
  "period": "2023-11-18",
  "comparison_period": "2023-11-17",
  "metrics": {
    "GMV": {"value": 985800, "change": -0.07},  # -7%
    "Заказы": {"value": 1778, "change": 0.04},  # +4%
    "Средний чек (AOV)": {"value": 554.44, "change": -0.106},# -10.6%
    "Сессии": {"value": 51000, "change": 0.041},  # +4.1%
    "Конверсия (CR)": {"value": 0.03486, "change": -0.001}# -0.1%
  }
}

# 5. Почти без изменений / Стабильность (День)
daily_flat = {
  "period": "2023-11-19",
  "comparison_period": "2023-11-18",
  "metrics": {
    "GMV": {"value": 990000, "change": 0.004}, # +0.4%
    "Заказы": {"value": 1785, "change": 0.004}, # +0.4%
    "Средний чек (AOV)": {"value": 554.62, "change": 0.0},  # 0%
    "Сессии": {"value": 51100, "change": 0.002}, # +0.2%
    "Конверсия (CR)": {"value": 0.03493, "change": 0.002} # +0.2%
  }
}  

# 6. Умеренный недельный рост
weekly_moderate_growth = {
  "period": "Неделя 46, 2023",
  "comparison_period": "Неделя 45, 2023",
  "metrics": {
    "GMV": {"value": 7800000, "change": 0.09},  # +9%
    "Заказы": {"value": 13500, "change": 0.07},  # +7%
    "Средний чек (AOV)": {"value": 577.78, "change": 0.019}, # +1.9%
    "Сессии": {"value": 450000, "change": 0.05},  # +5%
    "Конверсия (CR)": {"value": 0.03, "change": 0.019} # +1.9%
  }
}

# 7. Недельное падение с аномалией в трафике (GMV/Заказы вниз, Сессии вверх)
weekly_decline_traffic_anomaly = {
  "period": "Неделя 47, 2023",
  "comparison_period": "Неделя 46, 2023",
  "metrics": {
    "GMV": {"value": 7332000, "change": -0.06}, # -6%
    "Заказы": {"value": 12420, "change": -0.08}, # -8%
    "Средний чек (AOV)": {"value": 590.34, "change": 0.022}, # +2.2%
    "Сессии": {"value": 472500, "change": 0.05},  # +5% (аномалия!)
    "Конверсия (CR)": {"value": 0.0263, "change": -0.124} # -12.4%
  }
}

# 8. Фокус на конверсии (День) (GMV/Заказы стабильны, Сессии упали -> CR выросла)
daily_conversion_focus = {
  "period": "2023-11-20",
  "comparison_period": "2023-11-19",
  "metrics": {
    "GMV": {"value": 995000, "change": 0.005}, # +0.5%
    "Заказы": {"value": 1790, "change": 0.003}, # +0.3%
    "Средний чек (AOV)": {"value": 555.87, "change": 0.002}, # +0.2%
    "Сессии": {"value": 48545, "change": -0.05}, # -5%
    "Конверсия (CR)": {"value": 0.03687, "change": 0.056} # +5.6%
  }
}

# 9. Рост за счет AOV (Неделя) (GMV умеренно вырос, Заказы слегка упали)
weekly_aov_driven_growth = {
  "period": "Неделя 48, 2023",
  "comparison_period": "Неделя 47, 2023",
  "metrics": {
    "GMV": {"value": 7845240, "change": 0.07},  # +7%
    "Заказы": {"value": 12234, "change": -0.015}, # -1.5%
    "Средний чек (AOV)": {"value": 641.25, "change": 0.086}, # +8.6%
    "Сессии": {"value": 470000, "change": -0.005}, # -0.5%
    "Конверсия (CR)": {"value": 0.0260, "change": -0.01} # -1%
  }
}

# 10. Резкий всплеск (например, старт большой распродажи) (День)
daily_promo_spike = {
  "period": "2023-11-24", # Черная пятница :)
  "comparison_period": "2023-11-23",
  "metrics": {
    "GMV": {"value": 2400000, "change": 0.6},  # +60%
    "Заказы": {"value": 3100, "change": 0.55},  # +55%
    "Средний чек (AOV)": {"value": 774.19, "change": 0.032}, # +3.2%
    "Сессии": {"value": 80000, "change": 0.4},  # +40%
    "Конверсия (CR)": {"value": 0.03875, "change": 0.107} # +10.7%
  }
}

In [10]:
print(generate_kpi_mistral(daily_strong_growth))

В течение прошлого дня наблюдался сильный рост по всем фронтам бизнеса:

* GMV выросла на 25,0%
* Заказы также показали значительный рост в +20,0%
* Средний чек (AOV) увеличился на 4,2%
* Сессии продолжили расти и в этом случае на 18,0%
* Конверсия (CR) со стороны посетителей также показала смешанную динамику, заметно выросла на 1,7%.

Примечательно, что все эти показатели были значительно больше в течение предыдущего дня. Предлагается отложить месячный анализ, чтобы сравнить эту динамику со средними значениями за весь месяц и установить какие из изменений являются стабильными или временными тенденциями.


In [16]:
print(generate_kpi_mistral(daily_strong_decline))

В течение последнего дня наблюдается снижение основных KPI:

* GMV упала на 22,0% в сравнении с предыдущим днем.
* Количество заказов сократилось на 18,0%.
* Средний чек (AOV) уменьшился на 4,9%.
* Сессии и конверсия (CR) также показали снижение на 15,0% и 3,5%, соответственно.

Требуется проведение дополнительного анализа данных и принимать меры для увеличения GMV и количества заказов в ближайшее время.


In [17]:
print(generate_kpi_mistral(daily_promo_spike))

В указанный период отмечено усиленное рост по всем основным индикаторам:

* GMV - +60,0%
* Заказы - +55,0%
* Средний чек (AOV) - +3,2%
* Сессии - +40,0%
* Конверсия (CR) - +10,7%

Возможно, повышение среднего чека и конверсии связано с активной промоакционной кампанией. Необходимо подробнее изучить данные для уточнения тенденций и наладить соответствующие меры корректировки, если это необходимо.


In [22]:
print(generate_kpi_mistral(weekly_aov_driven_growth))

В неделе 48, 2023 годовой доход (GMV) увеличился на 7,0%, тогда как количество заказов упало на 1,5%. При этом средний чек (AOV) вырос на 8,6%, в то время как сессии сократились на 0,5% и конверсия (CR) снизилась на 1,0%. В целом, это означает, что потребители делают большие заказы, но их количество уменьшается. Необходимо продолжить анализ для обнаружения причин данного сдвига и принятия соответствующих мер для коррекции этого процесса.


In [23]:
print(generate_kpi_mistral(daily_mixed_gmv_down_orders_up))

Отчёт по KPI за период с 2023-11-17 по 2023-11-18:

GMV: Сильный спад (-7.0%)
Заказы: Слабое увеличение (+4.0%)
Средний чек (AOV): Существенное сокращение (-10.6%)
Сессии: Вторичный рост (+4.1%)
Конверсия (CR): Незначительное снижение (-0.1%)

В связи с данными предлагается провести анализ поведения пользователей и рассмотреть варианты оптимизации контента для улучшения среднего чека (AOV) и конверсии (CR).


### Полагаю, что результат можно улучшить, если указать конкретный построчный шаблон с вариантами классификации (Скачок, спад, смешанная статистика). Можно попробовать внутри промпта спрашивать модель вместо того, чтобы указывать ей.